# Лабораторная работа №5  
## Численное интегрирование (Ньютона–Котеса + Гаусс)

### Цель
Для таблично заданной функции на отрезке $[x_0, x_n]$ вычислить приближённое значение интеграла  

$$
I = \int_{x_0}^{x_n} f(x)\,dx
$$

методами Ньютона–Котеса (прямоугольники, трапеции, Симпсон) и методом Гаусса (2/3/4 точки).

---

## Исходные данные

| x    | f(x)     |
|------|----------|
| 0.00 | 1.000000 |
| 0.25 | 0.979915 |
| 0.50 | 0.927295 |
| 0.75 | 0.858001 |
| 1.00 | 0.785398 |
| 1.25 | 0.716844 |
| 1.50 | 0.655196 |
| 1.75 | 0.600943 |
| 2.00 | 0.553574 |

Шаг сетки: $h = 0.25$, число отрезков $n = 8$.

---

## Формулы

### Метод прямоугольников

**Левые:**

$$
I \approx \sum_{i=0}^{n-1}(x_{i+1}-x_i) \cdot f(x_i)
$$

**Правые:**

$$
I \approx \sum_{i=0}^{n-1}(x_{i+1}-x_i) \cdot f(x_{i+1})
$$

**Средние:**

$$
I \approx \sum_{i=0}^{n-1}(x_{i+1}-x_i) \cdot f\left(\frac{x_i+x_{i+1}}{2}\right)
$$

> Для средних прямоугольников и Гаусса нужны значения $f(x)$ **между узлами**, поэтому ниже используется натуральный кубический сплайн по табличным данным.

### Метод трапеций

$$
I \approx h \left( \frac{f_0}{2} + f_1 + \dots + f_{n-1} + \frac{f_n}{2} \right)
$$

### Метод Симпсона

$$
I \approx \frac{h}{3} \left( f_0 + f_n + 4\sum f_{2k-1} + 2\sum f_{2k} \right)
$$

### Метод Гаусса–Лежандра ($m = 2, 3, 4$)

$$
\int_a^b f(x)\,dx \approx \frac{b-a}{2} \sum_{k=1}^{m} w_k \cdot f\left( \frac{a+b}{2} + \frac{b-a}{2} \cdot t_k \right)
$$


In [1]:
import numpy as np
import pandas as pd

In [2]:
x = np.array([0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0], dtype=float)
y = np.array([1.000000, 0.979915, 0.927295, 0.858001, 0.785398, 0.716844, 0.655196, 0.600943, 0.553574], dtype=float)

def get_data():
    return x.copy(), y.copy()

In [3]:
def _uniform_step(x: np.ndarray, tol: float = 1e-12) -> float:
    x = np.asarray(x, float)
    h = np.diff(x)
    if np.any(h <= 0):
        raise ValueError("x должен быть строго возрастающим")
    if np.max(np.abs(h - h[0])) > tol:
        raise ValueError("нужна равномерная сетка (постоянный шаг)")
    return float(h[0])

# 1) Ньютона–Котеса: прямоугольники / трапеции / Симпсон

def rectangles(x: np.ndarray, y: np.ndarray, kind: str = "left") -> float:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    h = _uniform_step(x)

    kind = kind.lower()
    if kind == "left":
        return h * float(np.sum(y[:-1]))
    if kind == "right":
        return h * float(np.sum(y[1:]))
    raise ValueError("kind должен быть 'left' или 'right'")

def trapezoid(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    h = _uniform_step(x)
    return h * float(0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])

def simpson(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    h = _uniform_step(x)
    n = len(x) - 1
    if n % 2 != 0:
        raise ValueError("для Симпсона нужно чётное число отрезков")
    s_odd = np.sum(y[1:-1:2])
    s_even = np.sum(y[2:-1:2])
    return (h / 3.0) * float(y[0] + y[-1] + 4.0 * s_odd + 2.0 * s_even)

In [4]:
# 2) Сплайн (натуральный кубический): для значений между узлами

def spline_build_M(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    n = len(x)
    h = np.diff(x)

    A = np.zeros((n, n), float)
    rhs = np.zeros(n, float)

    # натуральные ГУ: M0 = Mn = 0
    A[0, 0] = 1.0
    A[-1, -1] = 1.0

    for i in range(1, n - 1):
        A[i, i - 1] = h[i - 1]
        A[i, i] = 2.0 * (h[i - 1] + h[i])
        A[i, i + 1] = h[i]
        rhs[i] = 6.0 * ((y[i + 1] - y[i]) / h[i] - (y[i] - y[i - 1]) / h[i - 1])

    return np.linalg.solve(A, rhs)

def spline_eval(xn: np.ndarray, yn: np.ndarray, M: np.ndarray, xq) -> np.ndarray:
    xn = np.asarray(xn, float)
    yn = np.asarray(yn, float)
    M = np.asarray(M, float)
    xq = np.asarray(xq, float)

    idx = np.searchsorted(xn, xq) - 1
    idx = np.clip(idx, 0, len(xn) - 2)

    xi = xn[idx]
    xi1 = xn[idx + 1]
    hi = xi1 - xi

    a = (xi1 - xq) / hi
    b = (xq - xi) / hi

    return (
        a * yn[idx] + b * yn[idx + 1]
        + ((a**3 - a) * M[idx] + (b**3 - b) * M[idx + 1]) * (hi**2) / 6.0
    )

def rectangles_midpoint(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x, float)
    y = np.asarray(y, float)
    h = _uniform_step(x)
    M = spline_build_M(x, y)
    mids = x[:-1] + 0.5 * h
    ym = spline_eval(x, y, M, mids)
    return h * float(np.sum(ym))

In [5]:
# 3) Гаусс–Лежандр (2/3/4 точки) по всему [a,b], f(x) через сплайн

_GAUSS = {
    2: (np.array([-1 / np.sqrt(3), 1 / np.sqrt(3)]), np.array([1.0, 1.0])),
    3: (np.array([-np.sqrt(3 / 5), 0.0, np.sqrt(3 / 5)]), np.array([5 / 9, 8 / 9, 5 / 9])),
    4: (
        np.array([-0.8611363115940526, -0.3399810435848563, 0.3399810435848563, 0.8611363115940526]),
        np.array([0.3478548451374539, 0.6521451548625461, 0.6521451548625461, 0.3478548451374539]),
    ),
}

def gauss_legendre(x: np.ndarray, y: np.ndarray, m: int) -> float:
    if m not in _GAUSS:
        raise ValueError("m должен быть 2, 3 или 4")

    x = np.asarray(x, float)
    y = np.asarray(y, float)

    t, w = _GAUSS[m]
    a, b = float(x[0]), float(x[-1])

    M = spline_build_M(x, y)
    xm = 0.5 * (a + b)
    xr = 0.5 * (b - a)

    xq = xm + xr * t
    fq = spline_eval(x, y, M, xq)

    return xr * float(np.sum(w * fq))

In [6]:
res = {
    "Прямоугольники (левые)": rectangles(x, y, "left"),
    "Прямоугольники (правые)": rectangles(x, y, "right"),
    "Прямоугольники (средние, сплайн)": rectangles_midpoint(x, y),
    "Трапеции": trapezoid(x, y),
    "Симпсон": simpson(x, y),
    "Гаусс 2 точки (сплайн)": gauss_legendre(x, y, 2),
    "Гаусс 3 точки (сплайн)": gauss_legendre(x, y, 3),
    "Гаусс 4 точки (сплайн)": gauss_legendre(x, y, 4),
}

pd.DataFrame({"Метод": list(res.keys()), "I": list(res.values())})
for k, v in res.items():
    print(f"{k:32s} = {v:.10f}")

Прямоугольники (левые)           = 1.6308980000
Прямоугольники (правые)          = 1.5192915000
Прямоугольники (средние, сплайн) = 1.5761402964
Трапеции                         = 1.5750947500
Симпсон                          = 1.5760136667
Гаусс 2 точки (сплайн)           = 1.5840587035
Гаусс 3 точки (сплайн)           = 1.5754732210
Гаусс 4 точки (сплайн)           = 1.5753544807
